In [1]:
import pandas as pd
import json
from csv import reader, writer
import os
from copy import deepcopy

In [46]:
dir="../benchmark/sap"
# file="benchmark_v5_fixed_filtered.csv"
file="v5_test_benchmark.csv"
out_file="v5_test_benchmark.jsonl"
loio_map="en_loio_map.tsv"
use_loios = True
loio2id = {}
build_loio = True
corpus="extended_components.jsonl"
docdata = []

In [48]:
def extract_loio(d):
    return d['url'].split("/")[-1].replace(".html","")

if build_loio:
    loio2id = {}
    with open(os.path.join(dir,corpus),"r") as f:
        for line in f:
            docdata = json.loads(line)
            loio = extract_loio(docdata)
            if loio not in loio2id:
                loio2id[loio] = [docdata['document_id']]
            else:
                loio2id[loio].append(docdata['document_id'])

else:
    with open(os.path.join(dir, loio_map),"r") as f:
        r1 = reader(f, delimiter="\t")
        for row in r1:
            if row[1] not in loio2id:
                loio2id[row[1]] = [row[0]]
            else:
                loio2id[row[1]].append(row[0])


In [ ]:
[k for k,v in loio2id.items() if len(v)>1]

In [56]:
loio2id['9f301385d30f4c32924590548c85f164']

KeyError: '9f301385d30f4c32924590548c85f164'

In [54]:
r=reader(open(os.path.join(dir, file), "r"), delimiter=",")
data=[l for l in r]

In [55]:
res=[]
seen = {}
for d in data:
    d1 = deepcopy(d)
    if d1[0] not in seen:
        seen[d1[0]]=True
        if d1[0]=='Count':
            d1.append("relevant")
        else:
            o = []
            for i in range(6,9):
                if d1[i]!='':
                    o.append(d1[i].replace("loio", ""))
            d1.append(o)
        #print(d1)
        if use_loios:
            for loio in o:
                try:
                    res.extend(loio2id[loio])
                except KeyError as e:
                    print(f"Key {e} not found")
        else:
            res.append(d1)

Key '9f301385d30f4c32924590548c85f164' not found


In [35]:
loio2id

{'41de98cee211444f92aa9b83e161c892': 'sap_00015.txt',
 '444b844332ef4a45aaf685bf0097d184': 'sap_00016.txt',
 '12f364ef310240dea740e25d562ccfa6': 'sap_00017.txt',
 '523cc6ac6db3404a84bfef030a33a36b': 'sap_00019.txt',
 '78444165263648c39d1584c82af5eddc': 'sap_00020.txt',
 '35e1036bad844c96a50d6bd94879a2eb': 'sap_00021.txt',
 '36cae9181fa64927bf2aae55b1e00915': 'sap_00022.txt',
 '48394df2081545999d9bca6a9b3231db': 'sap_00024.txt',
 'a5965d9220354bcf8839d65fc6655e9e': 'sap_00027.txt',
 '8faf299951a9468ba868d1ea5378cd63': 'sap_00028.txt',
 'b1c5835867f0472393b50fa8c2663406': 'sap_00029.txt',
 '8f5ee825b93a4eb981c2940c410e009a': 'sap_00030.txt',
 '729b8d1cc22b43098da58162f2e6df31': 'sap_00032.txt',
 '78b26700d1e845b588fd30bb8d8bb127': 'sap_00033.txt',
 'a38e385b1e844fa292e452b15ee2b03e': 'sap_00034.txt',
 '9882be80307d4116883ce7d6f6c42525': 'sap_00035.txt',
 'a17a855f4c734ed79d4bcd4fcba189a7': 'sap_00039.txt',
 '08376fbd9f1240169260f3914bd1dea3': 'sap_00040.txt',
 'a44f89038bad4a6f95512da574

In [30]:
with open(os.path.join(dir, out_file), "w") as out:
    headers = res[0]
    for d in res[1:]:
        out.write(json.dumps({headers[i]:d[i] for i in range(0,len(res[0]))})+"\n")

In [24]:
headers

['Count',
 'index',
 'Category',
 'Gold Answer',
 'Product',
 'Question',
 'loio 1',
 'loio 2',
 'loio 3',
 'relevant']

In [26]:
headers[9]

'relevant'